personal use script: Wi9ZVFvPbf_grQ
secret:	vJwRLc9MhDypqom3bM6tALNgx14

PRAW stands for Python Reddit API Wrapper, so it makes it very easy for us to access Reddit data. First we connect to Reddit by calling the praw.Reddit function and storing it in a variable. I’m calling mine reddit. You should pass the following arguments to that function:

In [10]:
#!/usr/bin/env python3
from datetime import datetime as dt
import csv
import requests
import time
import urllib.request, json
from operator import itemgetter
from collections import Counter
from operator import truediv


#Load in array of teams
teamList=[
#          ['Min', '', 'Scale', 'Scale', 'rgba(0, 0, 0, 0)', 'rgba(0, 0, 0, 0)'],
         ['Buffalo', 'Bills', 'AFC', 'AFC East', '#00338D', '#C60C30'],
         ['Miami', 'Dolphins', 'AFC', 'AFC East', '#008D97', '#F5811F'],
         ['New England', 'Patriots', 'AFC', 'AFC East', '#0D254C', '#C80815'],
         ['New York', 'Jets', 'AFC', 'AFC East', '#0C371D', '#0D7537'],
         ['Baltimore', 'Ravens', 'AFC', 'AFC North', '#280353', '#000000'],
         ['Cincinnati', 'Bengals', 'AFC', 'AFC North', '#FB4F14', '#000000'],
         ['Cleveland', 'Browns', 'AFC', 'AFC North', '#FE3C00', '#22150C'],
         ['Pittsburgh', 'Steelers', 'AFC', 'AFC North', '#FFB612', '#00000 0'],
         ['Tennessee', 'Titans', 'AFC', 'AFC South', '#648FCC', '#C0001D'],
         ['Houston', 'Texans', 'AFC', 'AFC South', '#02253A', '#B31B34'],
         ['Jacksonville', 'Jaguars', 'AFC', 'AFC South', '#006778', '#9F792C'],
         ['Indianapolis', 'Colts', 'AFC', 'AFC South', '#003B7B', '#0C77EB'],
         ['Denver', 'Broncos', 'AFC', 'AFC West', '#002244', '#FB4F14'],
         ['Kansas City', 'Chiefs', 'AFC', 'AFC West', '#F2C800', '#B20032'],
         ['Oakland', 'Raiders', 'AFC', 'AFC West', '#C4C8CB', '#ffffff'],
         ['Los Angeles', 'Chargers', 'AFC', 'AFC West', '#13264B', '#C9AF74'],
         ['Dallas', 'Cowboys', 'NFC', 'NFC East', '#0D254C', '#C5CED6'],
         ['New York', 'Giants', 'NFC', 'NFC East', '#192F6B', '#CA001A'],
         ['Philadelphia', 'Eagles', 'NFC', 'NFC East', '#003B48', '#708090'],
         ['Washington', 'Redskins', 'NFC', 'NFC East', '#773141', '#000000'],
         ['Chicago', 'Bears', 'NFC', 'NFC North', '#03202F', '#DD4814'],
         ['Detroit', 'Lions', 'NFC', 'NFC North', '#006DB0', '#C5C7CF'],
         ['Green Bay', 'Packers', 'NFC', 'NFC North', '#203731', '#FFB612'],
         ['Minnesota', 'Vikings', 'NFC', 'NFC North', '#3B0160', '#F0BF00'],
         ['Atlanta', 'Falcons', 'NFC', 'NFC South', '#BD0D18', '#000000'],
         ['Carolina', 'Panthers', 'NFC', 'NFC South', '#0088CE', '#A5ACAF'],
         ['New Orleans', 'Saints', 'NFC', 'NFC South', '#D2B887','#000000'],
         ['Tampa Bay', 'Buccaneers', 'NFC', 'NFC South', '#89765F', '#000000'],
         ['Arizona', 'Cardinals', 'NFC', 'NFC West', '#9b2743', '#c9002f'],
         ['Los Angeles', 'Rams', 'NFC', 'NFC West', '#002244', '#866D4B'],
         ['San Francisco', '49ers', 'NFC', 'NFC West', '#AF1E2C', '#E6BE8A'],
         ['Seattle', 'Seahawks', 'NFC', 'NFC West', '#002244', '#69BE28'],
#          ['Max', '', 'Scale', 'Scale', 'rgba(0, 0, 0, 0)', 'rgba(0, 0, 0, 0)']
        ]

abreviations = {"Bucaneers": "bucs", "Jaguars": "jags", "Cardinals": 'cards', 'Browns': "Brownies"}

########################
with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=browns&aggs=created_utc&frequency=year') as url:
    data = json.loads(url.read().decode())
    yearMentionData = (data['aggs']['created_utc'])
    
years = []
for pair in yearMentionData:
    unixTime = pair['key']
    dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y')
    years.append(dateTime)
    # space for ordering

matrix=[years]

for team in teamList:
    with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=' + team[1] + '&aggs=created_utc&frequency=year') as url:
        data = json.loads(url.read().decode())
        teamData = (data['aggs']['created_utc'])
        teamDataDict = {}
        abrvTeamDataDict = {}
        
        for pair in teamData:
            unixTime = pair['key']
            dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y')        
            teamDataDict[dateTime] = pair['doc_count']
            

        if(team[1] in abreviations):
            with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=' + abreviations[team[1]] + '&aggs=created_utc&frequency=year') as url:
                abrvData = json.loads(url.read().decode())
                abrvTeamData = (abrvData['aggs']['created_utc'])
            
                for pair in abrvTeamData:
                    unixTime = pair['key']
                    dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y')        
                    abrvTeamDataDict[dateTime] = pair['doc_count']
                teamDataDict = dict(Counter(abrvTeamDataDict) + Counter(teamDataDict))

        yearCount = []
        for time in years:
            if time in teamDataDict :
                yearCount.append(teamDataDict[time])
            else:
                yearCount.append(0)
        team = team + yearCount
        
        #sets min and max value for parcoords plot.  Parcoords 
#         if team[0] == "Min":         
#             team[6:] = [0] * len(team[0:])
#         if team[0] == "Max":
#             team[6:] = [140000] * len(team[0:])
        
        matrix.append(team)
spaceYears = []
for year in years:
    spaceYears.append(" " + str(year))
        
matrix[0] = ['City', 'Team', 'Conference', 'Division', 'PrimaryColor', 'SecondaryColor'] + spaceYears

matrix = [matrix[0]] + sorted(matrix[1:], key=itemgetter(6))




sums = [0] * (len(matrix[0])-6)
for team in matrix[1:]:
    array2 = team[6:]
    for index, count in enumerate(array2):
        sums[index] += count




with open('nflYear.csv', mode='w') as teams:
    writer = csv.writer(teams, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in matrix:
        writer.writerow(row)

percentageMatrix = matrix       
for team in percentageMatrix[1:]:
    array2 = team[6:]
    team[6:]=[100*float(ai)/bi for ai,bi in zip(array2,sums)]

with open('nflYearPercent.csv', mode='w') as teams:
    writer = csv.writer(teams, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in percentageMatrix:
        writer.writerow(row)

##################################

with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=browns&aggs=created_utc&frequency=Month') as url:
    data = json.loads(url.read().decode())
    MonthMentionData = (data['aggs']['created_utc'])

Months = []
for pair in MonthMentionData:
    unixTime = pair['key']
    dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y-%m')
    Months.append(dateTime)
    # space for ordering

matrix=[Months]

for team in teamList:
    with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=' + team[1] + '&aggs=created_utc&frequency=Month') as url:
        data = json.loads(url.read().decode())
        teamData = (data['aggs']['created_utc'])
        teamDataDict = {}
        abrvTeamDataDict = {}

        for pair in teamData:
            unixTime = pair['key']
            dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y-%m')
            teamDataDict[dateTime] = pair['doc_count']


        if(team[1] in abreviations):
            with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=' + abreviations[team[1]] + '&aggs=created_utc&frequency=Month') as url:
                abrvData = json.loads(url.read().decode())
                abrvTeamData = (abrvData['aggs']['created_utc'])

                for pair in abrvTeamData:
                    unixTime = pair['key']
                    dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y-%m')
                    abrvTeamDataDict[dateTime] = pair['doc_count']
                teamDataDict = dict(Counter(abrvTeamDataDict) + Counter(teamDataDict))

        MonthCount = []
        for time in Months:
            if time in teamDataDict :
                MonthCount.append(teamDataDict[time])
            else:
                MonthCount.append(0)
        team = team + MonthCount

        #sets min and max value for parcoords plot.  Parcoords
#          if team[0] == "Min":
#              team[6:] = [0] * len(team[0:])
#          if team[0] == "Max":
#              team[6:] = [140000] * len(team[0:])

        matrix.append(team)
spaceMonths = []
for Month in Months:
    spaceMonths.append(" " + str(Month))

matrix[0] = ['City', 'Team', 'Conference', 'Division', 'PrimaryColor', 'SecondaryColor'] + spaceMonths

matrix = [matrix[0]] + sorted(matrix[1:], key=itemgetter(6))




sums = [0] * (len(matrix[0])-6)
for team in matrix[1:]:
    array2 = team[6:]
    for index, count in enumerate(array2):
        sums[index] += count




with open('nflMonth.csv', mode='w') as teams:
    writer = csv.writer(teams, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in matrix:
        writer.writerow(row)

percentageMatrix = matrix
for team in percentageMatrix[1:]:
    array2 = team[6:]
    team[6:]=[100*float(ai)/bi for ai,bi in zip(array2,sums)]

with open('nflMonthPercent.csv', mode='w') as teams:
    writer = csv.writer(teams, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in percentageMatrix:
        writer.writerow(row)

# ##############################################################

# with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=browns&aggs=created_utc&frequency=week') as url:
#     data = json.loads(url.read().decode())
#     weekMentionData = (data['aggs']['created_utc'])

# weeks = []
# for pair in weekMentionData:
#     unixTime = pair['key']
#     dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y-%m-%W')
#     weeks.append(dateTime)
# matrix=[weeks]

# for team in teamList:
#     with urllib.request.urlopen('https://api.pushshift.io/reddit/comment/search?before=1551228873&after=1262307661&subreddit=nfl&q=' + team[1] + '&aggs=created_utc&frequency=week') as url:
#         data = json.loads(url.read().decode())
#         teamData = (data['aggs']['created_utc'])
#         teamDataDict = {}

#         for pair in teamData:
#             unixTime = pair['key']
#             dateTime = dt.utcfromtimestamp(unixTime).strftime('%Y-%m-%W')
#             teamDataDict[dateTime] = pair['doc_count']

#         weekCount = []

#         for time in weeks:
#             if time in teamDataDict:
#                 weekCount.append(teamDataDict[time])
#             else:
#                 weekCount.append(0)
#         team = team + weekCount
#         matrix.append(team)


# matrix[0] = ['City', 'Team', 'Conference', 'Division', 'PrimaryColor', 'SecondaryColor'] + years


# with open('nflWeek.csv', mode='w') as teams:
#     writer = csv.writer(teams, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

#     for row in matrix:
#         writer.writerow(row)

print("done")

done
